In [12]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()
groq_api_key = os.getenv('GROQ_API_KEY')
llm = ChatGroq(model="gemma2-9b-it",groq_api_key=groq_api_key)

In [2]:
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

c:\Users\ashwi\OneDrive\Desktop\AI\langchain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ashwi\OneDrive\Desktop\AI\langchain\venv\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ashwi\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python a

KeyboardInterrupt: 

In [13]:
from langchain_openai import OpenAIEmbeddings

embedding=OpenAIEmbeddings(model="text-embedding-3-small")

In [2]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain



USER_AGENT environment variable not set, consider setting it to identify your requests.


In [10]:
loader=WebBaseLoader(
    web_path="https://medium.com/gopenai/building-a-multi-pdf-rag-chatbot-langchain-streamlit-with-code-d21d0a1cf9e5"
)
docs= loader.load()

In [8]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=700,chunk_overlap=300)

In [11]:
documents = text_splitter.split_documents(docs)

In [14]:
vector_store = Chroma.from_documents(documents=documents,embedding=embedding)
retriever=vector_store.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001657436F7F0>, search_kwargs={})

In [24]:
system_prompt= """
You are an assistant for question answering task
use the following pieces of retrieved context to answer the question.
If you don't know the answer please say I don't know.
Keep the answer concise.
{context}
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

In [25]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [ ]:
response = rag_chain.invoke({'input':'How to set Up the Conversational AI'})
response['answer']

{'input': 'How to set Up the Conversational AI',
 'context': [Document(metadata={'description': 'Talking to big PDF’s is cool. You can chat with your notes, books and documents etc. This blog post will help you build a Multi RAG Streamlit based web application to read, process, and interact with…', 'language': 'en', 'source': 'https://medium.com/gopenai/building-a-multi-pdf-rag-chatbot-langchain-streamlit-with-code-d21d0a1cf9e5', 'title': 'Building a Multi PDF RAG Chatbot: Langchain, Streamlit with code | by Paras Madan | GoPenAI'}, page_content='The app sets up a conversational AI using OpenAI’s GPT model. This AI is designed to answer questions based on the PDF content it has processed.- Conversation Chain: The AI uses a set of prompts to understand the context and provide accurate responses to user queries. If the answer to a question isn’t available in the text, the AI is programmed to respond with “answer is not available in the context,” ensuring that users do not receive incorre

In [27]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

system_prompt = """
Give chat history and the latest question answer which might give reference context in the chat history.
formulate a standalone question which can be understood. without the chat history, do not answer the question.
just reformulate it if needed and otherwise return it as is.
"""

q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}")
    ]
)

In [28]:
history_aware_retriever=create_history_aware_retriever(llm,retriever,q_prompt)

In [29]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)

In [30]:
rag_chain = create_retrieval_chain(history_aware_retriever,question_answer_chain)

In [ ]:
response = rag_chain.invoke({'input':'How to set Up the Conversational AI','chat_history':get_chat})
response['answer']

"The app uses OpenAI's GPT model to answer questions based on processed PDF content. \n"

In [32]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

data_store = {}

def get_chat_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in data_store:
        data_store[session_id] = ChatMessageHistory()
    return data_store[session_id]

In [52]:
history=[]

In [53]:
response = rag_chain.invoke({'input':'How to set Up the Conversational AI','chat_history':history})
response['answer']

"The app uses OpenAI's GPT model to answer questions based on processed PDF content. \n"

In [55]:
from langchain_core.messages import HumanMessage,AIMessage
history.extend(
    [
        HumanMessage(content="How to set Up the Conversational AI"),
        AIMessage(content="The app uses OpenAI's GPT model to answer questions based on processed PDF content.  It uses a conversation chain with prompts for context understanding and accuracy")
    ]
)
history

[HumanMessage(content='How to set Up the Conversational AI', additional_kwargs={}, response_metadata={}),
 AIMessage(content="The app uses OpenAI's GPT model to answer questions based on processed PDF content.  It uses a conversation chain with prompts for context understanding and accuracy", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='How to set Up the Conversational AI', additional_kwargs={}, response_metadata={}),
 AIMessage(content="The app uses OpenAI's GPT model to answer questions based on processed PDF content.  It uses a conversation chain with prompts for context understanding and accuracy", additional_kwargs={}, response_metadata={})]

In [57]:
response = rag_chain.invoke({'input':'tell me more about it','chat_history':history})


In [58]:
response

{'input': 'tell me more about it',
 'chat_history': [HumanMessage(content='How to set Up the Conversational AI', additional_kwargs={}, response_metadata={}),
  AIMessage(content="The app uses OpenAI's GPT model to answer questions based on processed PDF content.  It uses a conversation chain with prompts for context understanding and accuracy", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How to set Up the Conversational AI', additional_kwargs={}, response_metadata={}),
  AIMessage(content="The app uses OpenAI's GPT model to answer questions based on processed PDF content.  It uses a conversation chain with prompts for context understanding and accuracy", additional_kwargs={}, response_metadata={})],
 'context': [Document(metadata={'description': 'Talking to big PDF’s is cool. You can chat with your notes, books and documents etc. This blog post will help you build a Multi RAG Streamlit based web application to read, process, and interact with…', 'language': 'en

In [59]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
)

In [61]:
config = {"configurable":{"session_id":"chat2"}}
conversational_rag_chain.invoke({"input":"What is Conversation Chain?"},config=config)

{'input': 'What is Conversation Chain?',
 'chat_history': [],
 'context': [Document(metadata={'description': 'Talking to big PDF’s is cool. You can chat with your notes, books and documents etc. This blog post will help you build a Multi RAG Streamlit based web application to read, process, and interact with…', 'language': 'en', 'source': 'https://medium.com/gopenai/building-a-multi-pdf-rag-chatbot-langchain-streamlit-with-code-d21d0a1cf9e5', 'title': 'Building a Multi PDF RAG Chatbot: Langchain, Streamlit with code | by Paras Madan | GoPenAI'}, page_content='The app sets up a conversational AI using OpenAI’s GPT model. This AI is designed to answer questions based on the PDF content it has processed.- Conversation Chain: The AI uses a set of prompts to understand the context and provide accurate responses to user queries. If the answer to a question isn’t available in the text, the AI is programmed to respond with “answer is not available in the context,” ensuring that users do not re

In [64]:
config = {"configurable":{"session_id":"chat2"}}
conversational_rag_chain.invoke({"input":"What is conversation chain?"},config=config)

{'input': 'What is conversation chain?',
 'chat_history': [HumanMessage(content='What is Conversation Chain?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="The Conversation Chain is a set of prompts used by the AI to understand the context of a user's question and provide an accurate response. \n", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is question did I ask?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='What is the purpose of the tool? \n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Which chain did I asked?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't know. \n", additional_kwargs={}, response_metadata={})],
 'context': [Document(metadata={'description': 'Talking to big PDF’s is cool. You can chat with your notes, books and documents etc. This blog post will help you build a Multi RAG Streamlit based web application to read, process, and interact with…'